<table>
  <tr>
    <td><p style="font-size:45px; color: #55BBD2">Analysis of light microscopy images in Python</p></td>
    <td><img src="ressources/lmb_logo.svg" alt="LMB Logo" width="500" height="600" align="right"></td>
  </tr>
</table>
<table>
  <tr>
    <td><p style="font-size:15px; color: #55BBD2">Version: February 2024</p></td>
  </tr>
</table>

# Introduction to the course

The course aims to introduce you to the essential knowledge to image processing and analysis of microscopic data in Python. 

At the end of this course, we hope the course attendees will be able to:
- understand and establish an image analysis workflow
- run a Python notebook
- get familiar with relevant Python packages
- load and read microscopic image using the package AICSImageOI
- filter the noise of an image
- segment cells using watershed segmentation
- segment cells using cellpose
- extract relevant information from an image using the module regionprops and save the results in an excel sheet
- track a particle and trace the trajectories of the particles

To tackle each of those topics, we will have three analysis problem. 
To run the Python notebook, select Kernel and choose the environment 'imaging'.

## Downloading data

Let us first download some data while looking at how to load a package. A package contains functions or modules that are ready to use.


In [ ]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
from pathlib import Path

def download_and_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

url = 'https://cloud.mrc-lmb.cam.ac.uk/s/AegYyMp3ajfaZME/download/data.zip'
data_folder = Path('./data')
download_and_unzip(url, data_folder)

print(f'List of files in the {data_folder} folder')
for x in (data_folder).glob('*'):
    print(f' {str(x.name)}')
 

# First analysis problem

## Measurement in a region of interest (ROI)

![title](ressources/data1.png)

In this first analysis problem, let say we are interested in what is happening in each channel within the actin region (Channel 1) that surround each nucleus (Channel 3). 

Our region of interest (ROI) is therefore defined by the region delimited by the actins in Channel 1 that have as a "seed" the nuclei in Channel 3. 

We want to measure:
- the maximal intensity in Channel 0 within this ROI,
- the median intensity in each of the ROI of Channel 1,  

The workflow of the analysis is therefore as follows:

<img src="ressources/workflow/workflow1.png" alt="drawing" width="800"/>

As an extra exercice after the course, you can also measure the number of dots in Channel 2 in each of the ROI. <br>

### Loading microscopy image data

For reading various type of microscopy data we suggest AICSImageIO.

[AICSImageIO](https://github.com/AllenCellModeling/aicsimageio) is a Python package that enable an unified and standardized reading of microscopic images in different formats. This package will use Python reader if possible or use Bio-Formats as a backup.

[Bio-Formats](http://www.openmicroscopy.org/bio-formats/) is a software tool for reading and writing life science image data using standardized, open formats. At the moment, it can supports 162 formats such as nd2, tifffile, lsm, etc. 

In [ ]:
from aicsimageio import AICSImage

image = AICSImage(data_folder / 'airyscan-4colors.tif')

The image and its metadata are now loaded into the variable "image". Next, we read the image and the metadata. 

### Metadata

Metadata stores information describing the data. It includes the number of channels in the images, the name of each channel and emission wavelength associated with each channel, the voxel size, and many other acquisition parameters.



In [ ]:
image.data
image.metadata
image.physical_pixel_sizes
image.shape
image.dims
image.channel_names

<div class="alert alert-success">

#### Exercise 

In the next cell, get familiar with the contents of the variable "image" and the metadata by printing the values stored in it one by one and understand the outputs. 
    
Hint: use the function print()
    
</div>

### Image dimension and pixel sizes

Next, we will check the different dimensions of our image. To illustrate this, we need to know what a dictionnary is as it is required here.

In [ ]:
image.dims

In [ ]:
for dim, element in enumerate(image.dims.order):
    print(f'Array axis {dim} is {element} with size {image.dims[element][0]}')

print(f'The name of the channels are {image.channel_names}')
print(f'The pixel size in X is {image.physical_pixel_sizes}')
print(f'The pixel size in X is {image.physical_pixel_sizes.X:.4f} microns')
print(f'The pixel size in Y is {image.physical_pixel_sizes.Y:.4f} microns')

### What is an image?

A digital image is a finite numeric representation of the intensity values. It is composed of picture elements known as pixels. 

In Python, it can be manipulated as a N-dimensional array using the class numpy.ndarray.

In [ ]:
# import the numpy package used for manipulating images
import numpy as np

# get the array of pixels intensity as a numpy object
data = image.data

# Print the type of the array
print('The data is a ', type(data)) 

# Print the dimension of the array
print('The array has the following shape', data.shape) 

# Print the physical pixel sizes and the types
pixel = image.physical_pixel_sizes
print('The physical pixel sizes are', pixel)
print('The pixel type is a', type(pixel)) 


Let us now display an image. 


In [ ]:
# import matplotlib for loading images
import matplotlib.pyplot as plt

# Let's display the second channel as a 2D image in a figure
plt.imshow(data[0,1,0,:,:], cmap='gray')

<div class="alert alert-success">
       
#### Exercise 

Display the second and third channels in the image and change the colormap cmap value to 'hot'.
    
Hint: use the same function that in the previous cell.
</div>

Now let's display all the channels of the image in a subplot using a `for` loop:

In [ ]:
# import matplotlib for displaying images
import matplotlib.pyplot as plt

# compute the number of channels
num_channels = image.shape[1]

# Display the image for each channel
fig, ax = plt.subplots(1, num_channels, figsize=(16,4))
for k  in range(num_channels):    
    ax[k].imshow(image.data[0,k,0], cmap='hot')
    ax[k].set_axis_off()
    ax[k].set_title(image.channel_names[k])    

In the following cell, we crop a part of the image, display it in a figure and overlay the pixel values.

In [ ]:
# Crop and downsample the array by a factor of 5
crop = image.data[0, 0, 0, 850:950:5, 900:1000:5] 

# Create a figure with axes
fig, ax = plt.subplots(figsize=(5, 5))

# Display the downsampled array in the figure
plt.imshow(crop, cmap='gray')

# Add the values of the pixel intensity
for i in range(crop.shape[0]):
    for j in range(crop.shape[1]):
        c = 'white' if crop[i, j] < 5 else 'black'
        ax.text(j, i, str(int(crop[i, j])), color=c, ha='center', va='center', size=10)
ax.axis("off")
plt.title('Pixel values');

<div class="alert alert-success">
       
#### Exercise 
    
Modify the previous cell to

1. select a different region within the image and display an overlay of the image with the pixel values,
2. check again the size of the image to be sure of not going out of bounds,
3. choose an appropriate downsampling factor,
4. adjust the figsize for a proper and visible display if necessary.
    
</div>

## Image filtering and noise

Filtering is a technique for modifying or enhancing an image. We can filter an image to emphasize certain features or remove other features. Noise filtering is an example of image filtering.

Noise in microscopic image is unavoidable during image acquisition. There are three main categories of noise: 
1. Photon noise, also called shot noise: acquired due to the fluctuation of photon emission to the detector. This is the most dominant noise. It is signal dependent and follows a Poisson distribution. 
2. Dark current or dark noise: accumulated over time due to thermal change as the detector heats up. It is unavoidable but negligible compared to photon noise and is not photon dependent. It follows Poisson distribution. 
3. Readout noise: digital noise acquired during the electronic quantisation of photons. It follows a Gaussian distribution where the standard deviation is constant and the mean is 0.

Dominant noise in fluorescence microscopy follows either a Gaussian distribution or Poisson distribution or both but mostly both. Gaussian noise is additive while Poisson noise is signal dependent. A correct removal of those two different types of noise may therefore differ. Firstly, we will look at how those noise affect the image differently. 

### Simulating noisy images

Here we are going to apply our understanding of what an image is to simulate a new one. <br>
We will add Gaussian and Poisson noise into our image to see the difference in the effect. 

In creating the image, we will use the floor division // operator, which returns the integer value of the quotient. 

In [ ]:
import numpy as np

# Create a noise free test image
n = 100
nphotons = 5
data = np.zeros((n,n),dtype=float)+0.5
data[n//4-10:n//4+10, n//4-10:n//4+10] = nphotons
data[n//4-10:n//4+10, 3*n//4-10:3*n//4+10] = nphotons
data[3*n//4-10:3*n//4+10, n//4:3*n//4] = nphotons

# Add a Gaussian noise
mean = 0
stddev = 3
noisy_gauss_img = data + np.random.normal(mean, stddev, data.shape)

# Apply Poisson noise
noisy_poisson_img = np.random.poisson(data)

fig, axs = plt.subplots(1, 3)
axs[0].imshow(data, cmap='gray',vmin=-3,vmax=nphotons+3)
axs[0].set_title('Original image')
axs[0].set_axis_off()

axs[1].imshow(noisy_gauss_img, cmap='gray',vmin=-3,vmax=nphotons+3)
axs[1].set_title('With Gaussian noise')
axs[1].set_axis_off()

axs[2].imshow(noisy_poisson_img, cmap='gray',vmin=-3,vmax=nphotons+3)
axs[2].set_title('With Poisson noise')
axs[2].set_axis_off()

fig.set_tight_layout('tight')


### Noise filtering

There are different types of noise reduction such as smoothing filtering or convolution filtering, median filtering, or frequency filtering. 

As each pixel has a value which represents the image intensity at the spatial position of the pixel, we need to work on each pixel of the image to filter an image.

### Gaussian filter
A smoothing filter uses the principle of convolution to reduce the noise. A convolution process is similar to drawing. The kernel is the pencil that is used for the drawing. So the sharpness of the drawn image really depends on the width of the point of the pencil. In the case of a Gaussian filtering, the Gaussian distribution is used as a kernel. When used for noise filterning, the noise which should have a lower intensity value than the real signal gets smoothed out because of this convolution process.

In [ ]:
from skimage import filters

# Crop the image and select the first channel
crop = image.data[0, 0, 0, 850:950, 900:1000]

# Gaussian filtering of data with a Gaussian filter of a given sigma
sigma = 3
gaussian_filtered = filters.gaussian(crop.astype(float), sigma)

fig, axs = plt.subplots(1, 2)
vmin, vmax = crop.min(), crop.max()

axs[0].imshow(crop, vmin = vmin, vmax = vmax)                      # original image
axs[0].set_title('Noisy image')
axs[0].set_axis_off()

axs[1].imshow(gaussian_filtered, vmin = vmin, vmax = vmax)                # Gaussian filtered
axs[1].set_title('Gaussian filtered ($\sigma=$'+str(sigma)+')')
axs[1].set_axis_off()

plt.tight_layout()


<div class="alert alert-success">
       
#### Exercise 

In the previous cell, change the standard deviation sigma of the Gaussian kernel and observe the change in the Gaussian filtered image.
    
</div>

### Median filter
A different technique for noise filtering is a median filtering. With the median filter, each output pixel is computed as the median value of the input pixel under a chosen window. 

In [ ]:
from skimage import filters

# define the kernel
median_kernel = np.ones((5,5)) 

# the median kernel is introduced in the second argument of the function 
median_filtered = filters.median(crop, median_kernel) 

fig, axs = plt.subplots(1, 2)

# compute the min and max of the original image to keep the same display range
vmin, vmax = crop.min(), crop.max()

axs[0].imshow(crop, vmin = vmin, vmax = vmax)    
axs[0].set_title('Noisy image')
axs[0].set_axis_off()

axs[1].imshow(median_filtered,  vmin = vmin, vmax = vmax)    
axs[1].set_title('Median filtered')
axs[1].set_axis_off()

plt.tight_layout()

<div class="alert alert-success">

#### Exercise       

Change the size of the median kernel and observe the change in the median filtered image.
    
</div>

## Segmentation

Image segmentation is the process of partitioning an image into multiple distinct regions. In semantic segmentation, regions can represent objects of interests against background. They can then be represented as a binary mask. Several type of objects can be further encoded using a label image. Finally, instance segmentation aims at distinguishing different identities of objects of potentially the same class and can also be represented using a label image.

To segment, we need to find the masks which represent each region of interest (ROI). This can easily be done by simply thresholding the intensity of the image. By this, the mask is defined such that only intensity (pixel) values greater than the threshold are selected. If we stop at this stage, we may encounter a problem such that some values within the thresholded region may get discarded because the intensity value is lower than the threshold and the ROI won't be filled. In this case, we may need to find the edge or/and fill the mask region. Another issue that we may encounter is that the edges of the regions may overlap so they may be detected or segmented as one. One solution in this case is to use what is called watershed segmentation. This technique helps to detect the flow of the intensity and decide from there whether the regions form only one object or different objects. 

### Thresholding 

The simplest way of doing this is to use a manually selected threshold, where an intensity value is defined as a threshold.

In [ ]:
from skimage import filters
import matplotlib.pyplot as plt


nuclei = image.data[0,3,0,:,:] # the object we want to segment and set as markers of our final ROI
nuclei_flt = filters.gaussian(nuclei.astype(float), 10, preserve_range=True) 

threshold = 5 # change threshold value here to see what happens to the mask that is being displayed below
mask = nuclei_flt >= threshold    
plt.imshow(mask, cmap='gray')
plt.title('Thresholded image')
plt.axis('off')

Lets plot the contour of this mask over the image to judge the quality of the segmentation.

In [ ]:
from skimage import measure

contours = measure.find_contours(mask)

plt.imshow(nuclei)
# For each elements of the contours
for contour in contours:
    plt.plot(contour[:,1], contour[:,0])

plt.axis('off')
plt.title('Contours of the segmented image');

### Automated thresholding segmentation 

If a single threshold is not suitable for a collection of images, a thresholding algorithm can be used. One example of this is Otsu's method [Otsu's method](https://en.wikipedia.org/wiki/Otsu%27s_method). More thresholding algorithms can be found in the [skimage.filters package](https://scikit-image.org/docs/stable/api/skimage.filters.html). 

In [ ]:
from skimage import filters

# find the threshold value differentiating the signal from background
threshold = filters.threshold_otsu(nuclei_flt) 

# define the threshold mask and close it
nuclei_mask = nuclei_flt > threshold

fig, ax = plt.subplots(1,3)

ax[0].imshow(nuclei_flt)
ax[0].set_title('Thresholded image')
ax[0].set_axis_off()

ax[1].hist(nuclei.ravel(),log=True)
ax[1].plot([threshold, threshold], [0,5e6])
ax[1].set_title('Otsu threshold')
ax[1].set_aspect(5)

ax[2].imshow(nuclei_mask)
ax[2].set_title('Thresholded image')
ax[2].set_axis_off()

plt.tight_layout()

### Post processing of binary masks
The mask obtained after thresholding can be further processed using morphological operations.

In [ ]:
from skimage import morphology
from skimage import segmentation
from skimage import measure
from skimage import color

# remove artifacts connected to image border
only_large_objects = morphology.remove_small_objects(nuclei_mask, 10000)
cleared = morphology.closing(only_large_objects, morphology.square(35))
cleared = segmentation.clear_border(cleared)

# label image regions
nuclei_label = measure.label(cleared)

# label image to rgb 
nuclei_label_overlay = color.label2rgb(nuclei_label, image=10*nuclei, bg_label=0)

fig, ax = plt.subplots(2, 3, figsize=(8,5))

ax[0,0].imshow(nuclei_flt)
ax[0,0].set_title('Filtered Nuclei channel')
ax[0,0].set_axis_off()


ax[0,1].imshow(nuclei_mask)
ax[0,1].set_title('Otsu threshold')
ax[0,1].set_axis_off()

ax[0,2].imshow(only_large_objects)
ax[0,2].set_title('Large Objects')
ax[0,2].set_axis_off()

ax[1,0].imshow(cleared)
ax[1,0].set_title('Cleared mask')
ax[1,0].set_axis_off()

ax[1,1].imshow(nuclei_label)
ax[1,1].set_title('Label image')
ax[1,1].set_axis_off()

ax[1,2].imshow(nuclei_label_overlay)
ax[1,2].set_title('Mask and image overlay')
ax[1,2].set_axis_off()

plt.tight_layout()

### Watershed-based segmentation
The segmentation of the nuclei is pretty straightforward because they are well separated. In some cases however, the objects we want to segment overlap, making it difficult to discern the different regions. In this case, we would need to add extra step into the image segmentation in order to discern the objects. Watershed segmentation is useful for this purpose. It is generally used for separating different objects. Watershed algorithm treats pixels values as a local topography (elevation) so the goal is to follow the flow of the elevation and find the watershed lines from which the edge of each of the regions are deduced. We use this principle to extract our previously defined ROIs: the actin region in Channel-1 having the two nuclei as markers and the seeds of the segmentation. 

In [ ]:
actin  = image.data[0,1,0,:,:] #  the channel we want to segment 

fig, axs = plt.subplots(1,2)
axs[0].imshow(actin)
axs[0].set_title('Actin channel')
axs[0].set_axis_off()

axs[1].imshow(nuclei)
axs[1].set_title('Nuclei channel')
axs[1].set_axis_off()

plt.tight_layout()

#### Mask in the actin channel
Create a mask on the actin using thresholding and morphological operations for closing and filling the mask. In the morphological operations, the erosion and dilation operations are used: <br>
- Erosion: removes small objects so only the big objects will remain in the image,
- Dilation: makes small objects more visible and fills in small holes in objects,
- Closing: a successive operation of those two operations dilation followed by an erosion.

The dynamic range of the intensity values of the actin image is low and it has a 8 bit depth. In this situation, it is very easy to clip out some intensity range or completely change the intensity range. This may yield to a huge error in the analysis. It is always advised to record an image at higher depth or increase the bit depth of the image. Here we convert the 8-bit image into 32-bit floating point.

In [ ]:
# convert 8-bit into 32-bit floating point
print('bit depth: ', actin.dtype)
print('(min, max) of the dynamic range of actin =', (np.min(actin), np.max(actin)))

actin_float = actin.astype(np.float32)
print('bit depth: ', actin_float.dtype)

print('(min, max) of the dynamic range of actin_float =', (np.min(actin_float), np.max(actin_float)))

In the following cell, firstly run the cell. Secondly, replace the input argument actin_float of the filters.gaussian by actin and observe the difference. Thirdly, replace it back to actin_float as the result from actin if of no interest to us.

In [ ]:
from scipy import ndimage as ndi
from skimage import filters

# smoothed the image using the Gaussian filter with the same sigma as before
smooth_actin = filters.gaussian(actin_float, sigma=3)
print('(min, max) of the dynamic range of smooth_actin =', (np.min(smooth_actin), np.max(smooth_actin)))

# find the optimal threshold
threshold = 0.5

# define the threshold mask
thresh_mask = smooth_actin > threshold

# fill holes to form the the actin_mask without performing the closing function
actin_mask = ndi.binary_fill_holes(thresh_mask).astype(int)

fig, axs = plt.subplots(1,4, figsize=(15, 15))
axs[0].imshow(actin)
axs[0].set_title('Original image')
axs[0].set_axis_off()

axs[1].imshow(smooth_actin)
axs[1].set_title('Gaussian filter')
axs[1].set_axis_off()

axs[2].imshow(thresh_mask)
axs[2].set_title('Thresholded image')
axs[2].set_axis_off()

axs[3].imshow(actin_mask)
axs[3].set_title('Fill holes')
axs[3].set_axis_off()

plt.tight_layout()


#### Distance transform and watershed
The next step will consists of computing the distance transform of the actin mask. Observe how the value of the distance inside the regions change with the spatial coordinates. 

In [ ]:
from scipy import ndimage as ndi
distance = ndi.distance_transform_edt(actin_mask) 

fig, ax = plt.subplots(1,3)

ax[0].imshow(actin_mask)
ax[0].set_title('Mask')
ax[0].set_axis_off()

im = ax[1].imshow(distance, cmap='jet')
ax[1].set_title('Distance transform')
ax[1].set_axis_off()

# insert the colorbar 
ax[2].set_position([0.64,0.4,0.025,0.2])
fig.colorbar(im, cax=ax[2])


Now we set the nuclei we segmented before as the markers or seeds of the segmentations. We use the markers to label the segmented regions in the actin region. 
After, we watershed the negative inverse of the distance transform of the actin using the defined markers and within the mask delimited by the actin. The negative inverse of the distance is used in the watershed function instead of the distance because the goal is to have the objects region as valleys not peak.  

In [ ]:
from skimage import segmentation

cell_label = segmentation.watershed(-distance, markers = nuclei_label, mask = actin_mask)

fig, ax = plt.subplots(1,4, figsize=(15, 15))

# create a RGB composite image to display the two channels (normalize channel in between 0 and 1)
rgb = np.stack([actin.astype(float) / actin.max(), 
                np.zeros(actin.shape), 
                nuclei.astype(float) / nuclei.max()], axis=2)

ax[0].imshow(rgb)
ax[0].set_axis_off()
ax[0].set_title('Actin and nuclei')

ax[1].imshow(distance, cmap='jet')
ax[1].set_title('Distance')
ax[1].set_axis_off()

ax[2].imshow(nuclei_label)
ax[2].set_title('Markers (seeds of segmentation)')
ax[2].set_axis_off()

ax[3].imshow(cell_label)
ax[3].set_title('Segmented ROIs')
ax[3].set_axis_off()

<div class="alert alert-success">

#### Exercise       

Copy the cell above and change the code to perform a watershed segmentation using the actin channel as a landscape image. Observe the results.
   
</div>


## Region properties

To measure information within the ROI masks that we have previously segmented, we use the regionprops function. It allows us to define a list of geometrical and intensity properties which can be measured in the images. 

This is a selection of useful properties supported by regionprops:
  - area
  - bbox
  - bbox_area
  - centroid
  - coords
  - equivalent_diameter
  - label
  - major_axis_length
  - max_intensity
  - mean_intensity
  - min_intensity
  - minor_axis_length
  - slice
  - solidity
  - eccentricity
  - orientation
  - perimeter

### Maximum Intensity


In [ ]:
import pandas as pd
from skimage import measure


signal = image.data[0,0,:,:].squeeze()
signal_table = measure.regionprops_table(cell_label, signal, properties=('label', 'intensity_max'))
results = pd.DataFrame(signal_table)

results

In [ ]:
import pandas as pd
from skimage import measure

signal = image.data[0,0,:,:].squeeze()

signal_props = measure.regionprops(cell_label, signal)
actin_props = measure.regionprops(cell_label, actin)

# Get the list of labels from the regions properties
labels = [p.label for p in signal_props]

# Get the maximum intensity for the properties
signal_max_intensity = [p.intensity_max for p in signal_props]
actin_max_intensity = [p.intensity_max for p in actin_props]

# Gather the results in a data frame (table)
results = pd.DataFrame({
    'Region label': labels, 
    'Signal [ch0] maximum intensity': signal_max_intensity,
    'Actin [ch1] maximum intensity': actin_max_intensity})

results


<div class="alert alert-success">

#### Exercise 

Use one of the methods above to measure the perimeter of the nuclei. Store the results in a variable called "perimeter_nuclei".
</div>

### Defining a custom measurement

The information about the median is currently not included in the regionprops function. Here we show how can we extract the intensity median for each ROI using a custom function. The median of a list is defined as the middle element of the list when the list is sorted. 

In [ ]:
# Define a custom function to measure median intensity
def median_intensity(mask, intensity):
    '''Median intenisty in a mask

    Parameters
    ----------
    mask : binary image
    intensity: intensity  image

    Returns
    -------
    The median intensity in the mask
    '''
    return np.median(intensity[mask])

actin_props = measure.regionprops_table(cell_label, actin, 
                                 properties = ('label','mean_intensity'), 
                                 extra_properties = (median_intensity,))

results = pd.DataFrame(actin_props)

results


## Save result
Export the result stored in the panda dataframe into a csv file

In [ ]:
results.to_csv('mean_median_results.csv')

<div class="alert alert-success">

#### Extra exercise 

At the end of this course, you can use the techniques described in this Python notebook and find the number of particles in Channel 2 of the data airyscan-4colors.tif within the labelled regions that we segmented above. 

Hint: <br>
1- use the blob detection used in the third analysis problem to find the position of the particles <br>
2- generate the locations of all the blobs <br>
3- measure the regionprops of the cell mask <br>
4- count how many blobs coordinates are there per segmented cell
</div>

# Second analysis problem

## Colocalization problem

![title](ressources/data2.png)

In this second analysis problem, we will focus on a different problem. Our data consists of an image with four channels. <br>
- Channel 0: Golgi marker,
- Channel 1: cargo protein,
- Channel 2: mitochondria,
- Channel 3: the nuclei of the cell. 

With a particular treatement, the cargo moves to the mitochondria. Without the mentioned treatement, the cargo goes to the Golgi. Our problem then consists of quantifying the relocation of the cargo to the mitochondria and its relocation to the Golgi. We also want to understand how the high (transfected) or low presence (not transfected) of the mitochondria in the cell affect this relocation of the cargo. Hence, the workflow of our image analysis is illustrated as follows : <br>

<img src="ressources/workflow/workflow2.png" alt="drawing" width="800"/>

## Load data

In [ ]:
from aicsimageio import AICSImage, readers

col_image = AICSImage(data_folder/'Zeiss1344.lsm', reader=readers.tiff_reader.TiffReader)
col_data = col_image.data
col_data_shape = col_data.shape

## Segment the cell using cellpose

Here we present a different technique for segmenting the cell. It uses a pretrained neural network for segmenting the image using two channels. <br> [Cellpose](https://cellpose.readthedocs.io/en/latest/) has been pretrained to detect cells or nuclei on a number of biological images.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import segmentation
from cellpose import models

# Get the pretrained model
model = models.Cellpose(gpu=False, model_type='cyto2')

# Evaluate the model
mask, flows, styles, diams = model.eval(col_data, channels=[2,3], diameter=400, cellprob_threshold=1.0)

# Display the results
fig, ax = plt.subplots(1,7, figsize=(15,5))
for k in range(4):
    ax[k].imshow(col_data[:,k,:,:,:].squeeze())
    ax[k].set_axis_off()
    ax[k].set_title('Ch0'+str(k))
ax[4].imshow(mask)
ax[4].set_axis_off()
ax[4].set_title('Labels')
ax[5].imshow(flows[0])
ax[5].set_axis_off()
ax[5].set_title('Flows')
ax[6].imshow(flows[2])
ax[6].set_axis_off()
ax[6].set_title('Probability')
fig.tight_layout()

## Regions properties

In [ ]:
from skimage import measure

props = measure.regionprops(mask)

## Using an interactive display
We will use the module 'interact' to view each labelled segmented region and observe if the segmentation was done as we expected it 

In [ ]:
from ipywidgets import interact
import numpy as np

def displayf(label):
    p = props[label]
    single_mask = np.zeros(col_data_shape[-2:])
    single_mask[p.coords[:,0], p.coords[:,1]] = 1
    
    plt.imshow(col_data[:,2,:,:].squeeze())                          # mitochondria
    plt.imshow(col_data[:,3,:,:].squeeze(), alpha=0.5 , cmap='grey') # nuclei

    plt.imshow(single_mask, alpha=0.2)

    return 

interact(displayf, label=(0, len(props)-1, 1)) # min, max, step


<div class="alert alert-success">

#### Exercise       

The cell segmentation was obtained after setting the parameters: particle size to 400 pixels and cellprob_threshold = 1.0. <br>
Observe the segmented regions change if you change the particle size or the cellprob_threshold.    
</div>

## Measure the colocalization coefficient 

The Pearson's correlation coefficient (PCC) is widely used to quantify the colocalization of one object $A$ into another object $B$. It can be calculated using the following formula [1]:

$$\text{PCC} = \frac{\sum_i\left( A_i - A_{aver} \right) \cdot \left( B_i - B_{aver} \right)}{\sqrt{\left[ \sum_i \left( A_i - A_{aver} \right)^2 \cdot \sum_i \left( B_i - B_{aver} \right)^2 \right]}}$$

$A$: cargo in this problem with $A_i$ being the intensity at voxel $i$ and $A_{aver}$ the average value intensity \
$B$: organelles (mitochondria or Golgi) with $B_i$ being the intensity at voxel $i$ and $B_{aver}$ the average value intensity

PCC values range from -1 to 1. We will use the function pearsonr from the scipy.stats module to compute it. We will calculate the PCC between the organelles, mitochondria or Golgi, and the cargo to quantify how much cargo relocates to each corresponding organelle for a given treatement done on the cell. We will observe how different the PCC values are in high presence (transfected) and low presence (not transfected) of mitochondria in each cell respectively. <br>

We will apply the following notations:
- PCC1: PCC between the cargo and Golgi 
- PCC2: PCC between the cargo and the mitochondria

Source [1]: Manders, E.M., Verbeek, F.J. and Aten, J.A., 1993. Measurement of co‐localization of objects in dual‐colour confocal images. Journal of microscopy, 169(3), pp.375-382.

In [ ]:
import scipy.stats
from skimage import measure
import pandas as pd

ColCoeff = []
intensity_threshold = 1e6

for p in props:
    ch00 = col_data[:,0,:,:].squeeze()[p.coords[:,0], p.coords[:,1]] # Golgi marker
    ch01 = col_data[:,1,:,:].squeeze()[p.coords[:,0], p.coords[:,1]] # Cargo protein
    ch02 = col_data[:,2,:,:].squeeze()[p.coords[:,0], p.coords[:,1]] # Mitochondria
    sum_mitochondria = ch02.sum()
    if sum_mitochondria >= intensity_threshold:
        c = 'Transfected'
    else:
        c = 'Not transfected'

    
    [r1, pv] = scipy.stats.pearsonr(ch00, ch01)
    [r2, pv] = scipy.stats.pearsonr(ch01, ch02)
    ColCoeff.append({'label':p.label,'PCC1':r1, 'PCC2':r2, 'Area':p.area, 'Integrated intensity':sum_mitochondria, 'Condition':c})

ColCoeff = pd.DataFrame.from_records(ColCoeff)
ColCoeff

## Statistical analysis

We firstly analyse the data which corresponds to the relocation of the cargo to the Golgi. The relocation of the cargo to the mitochondria will be done as an exercise.

### p-value

The p-value is calculated from our statistical test. It describes how likely we are to have found the set of observations we have if there is no relationship between our conditions of interest (amount of mitochondria in each cell)

In [ ]:
grp1 = ColCoeff[ColCoeff['Condition']=='Transfected']
grp2 = ColCoeff[ColCoeff['Condition']=='Not transfected']
pvalue = scipy.stats.mannwhitneyu(grp1['PCC1'],grp2['PCC1'])

print(pvalue)

## Display of the result
Display the PCC values, annotate the p-value and save the result into pdf file

In [ ]:
import seaborn as sns
from statannotations.Annotator import Annotator

conditions_list = ['Not transfected', 'Transfected']

plotting_parameters = {
    'data':    ColCoeff,
    'x':       'Condition',
    'y':       'PCC1',
    'order':   conditions_list,
}

ax = sns.violinplot(**plotting_parameters, color="0.9")
sns.stripplot(**plotting_parameters, jitter=True, size=2)

annotator = Annotator(ax, tuple([conditions_list]), **plotting_parameters)
annotator.set_pvalues([pvalue.pvalue])
annotator.configure(loc='outside')
annotator.annotate()

ax.set_title('Relocation of the \nCargo to the Golgi', y=1.0, pad=-10, c='red', horizontalalignment='center')

plt.savefig('PCC_Cargo_to_Golgi.pdf', format="pdf", bbox_inches="tight")


<div class="alert alert-success">

#### Exercise       

Display the result of the statistical analysis which corresponds to the relocation of cargo to the mitochondria. <br>
Make sure the title of the figure and the name under which the figure will be saved are updated.
   
</div>

# Third analysis problem

## Particle tracking
There are different methods we can use to track particles. Here we will use a spot detection, called blob detection. This technique aims at detecting regions in a digital image that differ in properties, such as brightness or color, compared to surrounding regions. The Difference of Gaussian (DoG) method is one technique to detect blobs. For each blob found (a spot in our case), the method returns its coordinates position and the standard deviation of the Gaussian kernel that detected the spot. 

The workflow of our image analysis is illustrated as follows : <br>

<img src="ressources/workflow/workflow3.png" alt="drawing" width="800"/>



In [ ]:
from aicsimageio import AICSImage

particles = AICSImage(data_folder / '181228_CDX2_9s_c48_n073.tif')
particles_data = particles.data
sz = particles_data.shape
print(sz)

## Interactive display
Now we are going to display the particles images, consisting of 105 frames, using the 'interact' module.

In [ ]:
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt

def displayf(t):
    myimg = np.squeeze(particles_data[t])

    return plt.imshow(myimg)

interact(displayf, t=(0, sz[0]-1, 1)) # min, max, step

## Detect each particle (spot) using blob_dog from the feature module

The blob_dog spot detection has few parameters that we need to care for in order to filter out what we consider particles and which ones are not. There are importants parameters that we will conder here: threshold, min_sigma and max_sigma. 

- threshold: The absolute lower bound for scale space maxima. Local maxima smaller than threshold are ignored. Reduce this to detect blobs with lower intensities. <br>
- min_sigma: The minimum standard deviation for Gaussian kernel.
- max_sigma: The maximum standard deviation for Gaussian kernel.

Further reading: https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_dog

We will save the positions of the detected particles in a panda dataframe.

In [ ]:
from skimage import feature 
import pandas as pd

particles_positions = []

for t in range(sz[0]):    
    coordinates = feature.blob_dog(particles_data[t].squeeze(), min_sigma=1, max_sigma=5, threshold=0.005)    
    particles_positions.append(pd.DataFrame({'x': coordinates[:,1], 'y': coordinates[:,0], 'w': coordinates[:,2], 'frame':t}))

particles_positions = pd.concat(particles_positions)
    
particles_positions

Overlay the particles (yellow crosses) positions on top of the images of the particles

In [ ]:
from ipywidgets import interact

def displayPos(t):
    myimg = np.squeeze(particles_data[t])
    coordinates = particles_positions.loc[particles_positions['frame']==t]

    plt.imshow(myimg)
    return plt.plot(coordinates['x'], coordinates['y'], '+', color='yellow', linewidth=0.01)   

interact(displayPos, t=(0, sz[0]-1, 1)) # min, max, step

<div class="alert alert-success">

#### Exercise       

The results we have here were optimized using threshold = 0.005, min_sigma = 1, and max_sigma = 5. You may find a better setting than these. <br> 
So go back to the spot detection cell and play with the values of those three parameters and observe how the results are changing accordingly.

   
</div>


## Particles trajectories
We are now going to trace the trajectories of the particles by linking the coordinates positions. <br> 
For this, we will use a Python package named 'trackpy'.


In [ ]:
import trackpy as tp

farthest_distance = 8 # farthest distance a particle can travel, larger value slows computation time
track_linked = tp.link(particles_positions, farthest_distance, memory=5) # value of memory corresponds to the number of frames that is being memorized, track_linked is a pandas dataframe
nb_particles = track_linked['particle'].unique().max() + 1
print('There are', nb_particles, 'particles')

plt.figure()
tp.plot_traj(track_linked)

<div class="alert alert-success">

#### Exercise       

Try to understand the code line which generates the number of particles. <br>
What do you think will happen to this number of particles if you change the allowed farthest distance a particle can travel? <br>

(Here we have farthest_distance = 8 initially)
   
</div>

## Additional constraints

But maybe we eventually want to remove particles that are only appearing in few frames?

Let say we won't consider the trajectories of particles that are appearing in less than 10 frames.

In [ ]:
import pandas as pd

l = track_linked['particle'].unique() 
allowed_track_length = 10
particles_id = [k for k in l if len(track_linked.loc[track_linked['particle']==k])>=allowed_track_length]

selected_tracks = pd.DataFrame([])

for t in range(len(particles_id)):
    coordinates = track_linked.loc[track_linked['particle']==particles_id[t]]
    selected_tracks = pd.concat([selected_tracks, coordinates])

selected_tracks

#### Display the selected tracks over the images of the particles to see how well we did 

In [ ]:
def displayTrack(t):
    myimg = np.squeeze(particles_data[t])

    plt.imshow(myimg)
    return tp.plot_traj(selected_tracks)

interact(displayTrack, t=(0, sz[0]-1, 1)) # min, max, step

<div class="alert alert-success">

#### Exercise       

Change the allowed_track_length and observe the number of trajectories displayed in the overlay display.
   
</div>

## Exporting tabular results

To save results in a csv file use the export function from pandas. <br>
The excelsheet will be save in your current directory. To check where is that directory, run the command pwd in a new cell and you will see the csv file be saved in that directory.


In [ ]:
pwd

In [ ]:
selected_tracks.to_csv('tracking_results.csv')

# Some more helpful tips

- scientific images are data, they can be compromised by inappropriate manipulations
- take good images:
    - don't oversaturate your data
    - use the full dynamic range when taking your images
- do not segment on the data you are measuring, use a housekeeping channel
- images that are compared to each other need to be processed and acquired in the same manner

